In [12]:
import pandas as pd
import os
from statsmodels.stats.anova import AnovaRM
from scipy.stats import ttest_rel
from google.colab import drive
from sklearn.linear_model import LinearRegression
import numpy as np

# Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

# URL da planilha do Google Sheets
sheet_id = '1p2-JXILLC0urC0oiTOHjbO1579CtXW2RfUIAK6R_z4c'
sheet_name = 'Sheet1'  # Substitua pelo nome da aba, se necessário
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

# Ler os dados da planilha usando pandas
try:
    df = pd.read_csv(url)
except Exception as e:
    raise ValueError(f"Erro ao ler a planilha: {e}")

# Verificar se os dados foram importados corretamente
if df is not None:
    # Padronizar os tipos de dados
    df = df.apply(pd.to_numeric, errors='coerce', downcast='integer')

    # Verificar se as colunas esperadas estão presentes
    expected_columns = ['Participante', 'Ausente_4', 'Ausente_12', 'Ausente_24', 'Presente_4', 'Presente_12', 'Presente_24']
    missing_columns = [col for col in expected_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"As seguintes colunas estão faltando na planilha: {missing_columns}")

    # Converter o DataFrame para formato longo
    long_df = pd.melt(df, id_vars=['Participante'],
                      value_vars=['Ausente_4', 'Ausente_12', 'Ausente_24', 'Presente_4', 'Presente_12', 'Presente_24'],
                      var_name='Condição', value_name='Tempo')

    # Separar os fatores
    long_df['Alvo'] = long_df['Condição'].apply(lambda x: 'Ausente' if 'Ausente' in x else 'Presente')
    long_df['Itens'] = long_df['Condição'].apply(lambda x: int(x.split('_')[1]))
    long_df = long_df.drop(columns=['Condição'])

    # Realizar a ANOVA
    anova = AnovaRM(long_df, depvar='Tempo', subject='Participante', within=['Alvo', 'Itens']).fit()

    # Calcular médias descritivas
    medias = long_df.groupby(['Alvo', 'Itens'])['Tempo'].mean().reset_index()
    tabela_medias = medias.pivot(index='Itens', columns='Alvo', values='Tempo')
    tabela_medias_texto = tabela_medias.to_string()

    # Testes Post Hoc: Testes t pareados com correção de Bonferroni
    itens_unicos = sorted(long_df['Itens'].unique())
    post_hoc_results = []
    for i in range(len(itens_unicos)):
        for j in range(i + 1, len(itens_unicos)):
            t_stat, p_value = ttest_rel(
                long_df[long_df['Itens'] == itens_unicos[i]]['Tempo'],
                long_df[long_df['Itens'] == itens_unicos[j]]['Tempo']
            )
            post_hoc_results.append({
                'Comparação': f"{itens_unicos[i]} vs {itens_unicos[j]}",
                't-Estatístico': t_stat,
                'p-Valor (não corrigido)': p_value
            })

    post_hoc_df = pd.DataFrame(post_hoc_results)

    # Verificar efeitos principais e interação
    efeito_principal_itens = anova.anova_table.loc['Itens', 'Pr > F']
    efeito_principal_alvo = anova.anova_table.loc['Alvo', 'Pr > F']
    interacao = anova.anova_table.loc['Alvo:Itens', 'Pr > F']

    if efeito_principal_itens < 0.05:
        resultado_itens = (f"Houve um efeito principal de número de itens, "
                          f"F(2,14) = {anova.anova_table.loc['Itens', 'F Value']:.2f}, p < 0.001.")
    else:
        resultado_itens = (f"Não foi encontrado o efeito principal de número de itens, "
                          f"F(2,14) = {anova.anova_table.loc['Itens', 'F Value']:.2f}, p = {efeito_principal_itens:.3f}.")

    if efeito_principal_alvo < 0.05:
        resultado_alvo = (f"Houve um efeito principal de presença do alvo, "
                          f"F(1,7) = {anova.anova_table.loc['Alvo', 'F Value']:.2f}, p = {efeito_principal_alvo:.3f}, "
                          f"com tempo médio de resposta mais curto quando o alvo estava presente.")
    else:
        resultado_alvo = (f"Não foi encontrado o efeito principal de presença do alvo, "
                          f"F(1,7) = {anova.anova_table.loc['Alvo', 'F Value']:.2f}, p = {efeito_principal_alvo:.3f}.")

    # Realizar uma regressão linear para calcular R²
    X = long_df[['Itens']]
    y = long_df['Tempo']

    reg = LinearRegression()
    reg.fit(X, y)
    r2 = reg.score(X, y)

    # Preparar o sumário em texto
    sumario_texto = """
Análise Detalhada do Experimento de Busca Visual
-------------------------------------------------

1. Estatísticas Descritivas:
{tabela_medias}

2. Resultados da ANOVA:
{anova_resultados}

3. Testes Post Hoc (t de Student pareado com Bonferroni):
{post_hoc}

4. Regressão Linear:
R² = {r2:.3f}

Conclusões:
- {resultado_itens}
- {resultado_alvo}
- Houve uma interação significativa entre a presença do alvo e o número de itens (p = {interacao:.3f}).
""".format(
        tabela_medias=tabela_medias_texto,
        anova_resultados=anova.summary().as_text(),
        post_hoc=post_hoc_df.to_string(index=False),
        r2=r2,
        resultado_itens=resultado_itens,
        resultado_alvo=resultado_alvo,
        interacao=interacao
    )

    # Criar o arquivo de sumário
    output_path = '/content/drive/My Drive/sumario.txt'
    with open(output_path, 'w') as f:
        f.write(sumario_texto)

    print(f"Arquivo de sumário salvo em: {output_path}")

Mounted at /content/drive
Arquivo de sumário salvo em: /content/drive/My Drive/sumario.txt
